# Демонстрация системы управления токенами TvDatafeed

Этот notebook демонстрирует новую систему автоматического управления auth token, которая решает проблему частых запросов капчи при использовании TvDatafeed.

## 1. Импорт библиотек и настройка логирования

In [ ]:
import logging
import time
import os
from tvDatafeed import TvDatafeed, Interval, TokenManager

# Настройка логирования для наглядности
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Библиотеки импортированы успешно")
print("📋 Логирование настроено для отслеживания работы с токенами")

## 2. Настройка учетных данных

**Важно:** Замените на ваши реальные учетные данные TradingView

In [ ]:
# Замените на ваши реальные учетные данные
username = 'YourTradingViewUserName'
password = 'YourTradingViewPassword'

# Файл для сохранения токена (можно настроить)
token_file = 'demo_token.json'

print(f"👤 Пользователь: {username}")
print(f"💾 Файл токена: {token_file}")

## 3. Тест TokenManager (прямое использование)

In [ ]:
print("🔧 Тестирование TokenManager...\n")

# Создаем менеджер токенов
token_manager = TokenManager(token_file)

# Проверяем наличие сохраненного токена
existing_token = token_manager.load_token(username)
if existing_token:
    print(f"🔍 Найден существующий токен: {existing_token[:20]}...")
    
    # Получаем информацию о токене
    token_info = token_manager.get_token_info()
    if token_info:
        print(f"📅 Создан: {token_info['created_at']}")
        print(f"⏰ Последнее использование: {token_info['last_used']}")
        print(f"📊 Возраст: {token_info['age_days']} дней")
        
        # Проверяем истечение
        is_expired = token_manager.is_token_expired(max_age_days=1)
        print(f"⚠️ Истек (>1 дня): {is_expired}")
else:
    print("❌ Сохраненный токен не найден")

print("\n" + "="*50)

## 4. Первое создание TvDatafeed (с сохранением токена)

In [ ]:
print("🚀 Создание TvDatafeed с автоматическим управлением токенами...\n")

# Создаем объект TvDatafeed с новой системой управления токенами
tv = TvDatafeed(
    username=username,
    password=password,
    token_file=token_file
)

print(f"✅ TvDatafeed создан успешно")
print(f"🔑 Текущий токен: {tv.token[:20] if tv.token != 'unauthorized_user_token' else 'Не авторизован'}...")

# Получаем информацию о токене
token_info = tv.get_token_info()
if token_info:
    print(f"📊 Информация о токене:")
    for key, value in token_info.items():
        print(f"   {key}: {value}")

print("\n" + "="*50)

## 5. Тест получения данных (нормальный сценарий)

In [ ]:
print("📈 Тестирование получения данных...\n")

try:
    # Получаем данные по AAPL
    print("📊 Запрос данных AAPL...")
    aapl_data = tv.get_hist('AAPL', 'NASDAQ', interval=Interval.in_daily, n_bars=10)
    
    if aapl_data is not None:
        print(f"✅ Данные получены успешно: {len(aapl_data)} записей")
        print(f"📅 Период: {aapl_data.index[0]} - {aapl_data.index[-1]}")
        print(f"💰 Последняя цена закрытия: ${aapl_data['close'].iloc[-1]:.2f}")
        
        # Показываем первые несколько записей
        print("\n📋 Первые 3 записи:")
        print(aapl_data.head(3))
    else:
        print("❌ Данные не получены")
        
except Exception as e:
    print(f"❌ Ошибка при получении данных: {e}")

print("\n" + "="*50)

## 6. Тест повторного создания TvDatafeed (загрузка сохраненного токена)

In [ ]:
print("🔄 Тестирование повторного создания TvDatafeed...\n")

# Создаем новый объект - должен загрузить сохраненный токен
tv2 = TvDatafeed(
    username=username,
    password=password,
    token_file=token_file
)

print(f"✅ Второй объект TvDatafeed создан")
print(f"🔑 Токен загружен: {tv2.token[:20] if tv2.token != 'unauthorized_user_token' else 'Не авторизован'}...")

# Сравниваем токены
if tv.token == tv2.token:
    print("✅ Токены идентичны - сохранение/загрузка работает корректно")
else:
    print("⚠️ Токены различаются - возможно, был создан новый токен")

print("\n" + "="*50)

## 7. Тест симуляции истекшего токена

In [ ]:
print("🧪 Симуляция работы с истекшим токеном...\n")

# Сохраняем оригинальный токен
original_token = tv.token
print(f"💾 Оригинальный токен: {original_token[:20]}...")

# Подменяем токен на заведомо недействительный
tv.token = "invalid_token_12345"
print(f"🔧 Установлен недействительный токен: {tv.token}")

print("\n📊 Попытка получения данных с недействительным токеном...")

try:
    # Пытаемся получить данные - система должна автоматически обновить токен
    msft_data = tv.get_hist('MSFT', 'NASDAQ', interval=Interval.in_daily, n_bars=5)
    
    if msft_data is not None:
        print(f"✅ Данные получены успешно после автоматического обновления токена!")
        print(f"🔑 Новый токен: {tv.token[:20]}...")
        print(f"📊 Получено записей: {len(msft_data)}")
        
        # Проверяем, обновился ли токен
        if tv.token != "invalid_token_12345" and tv.token != original_token:
            print("🎉 Токен был автоматически обновлен!")
        elif tv.token == original_token:
            print("🔄 Токен восстановлен до оригинального")
    else:
        print("❌ Данные не получены даже после попытки обновления токена")
        
except Exception as e:
    print(f"❌ Ошибка: {e}")
    print("⚠️ Автоматическое обновление токена не сработало")

print("\n" + "="*50)

## 8. Тест принудительного обновления токена

In [ ]:
print("🔄 Тестирование принудительного обновления токена...\n")

# Получаем информацию о текущем токене
old_token_info = tv.get_token_info()
if old_token_info:
    print(f"📊 Текущий токен создан: {old_token_info['created_at']}")

old_token = tv.token
print(f"🔑 Старый токен: {old_token[:20]}...")

# Принудительно обновляем токен
print("\n🔄 Выполняем принудительное обновление...")
refresh_success = tv.refresh_token()

if refresh_success:
    print("✅ Токен успешно обновлен!")
    print(f"🔑 Новый токен: {tv.token[:20]}...")
    
    # Получаем информацию о новом токене
    new_token_info = tv.get_token_info()
    if new_token_info:
        print(f"📊 Новый токен создан: {new_token_info['created_at']}")
    
    # Проверяем, что токен действительно изменился
    if tv.token != old_token:
        print("🎉 Токен успешно изменен!")
    else:
        print("⚠️ Токен остался тем же (возможно, старый токен еще валиден)")
else:
    print("❌ Не удалось обновить токен")

print("\n" + "="*50)

## 9. Тест множественных запросов (проверка стабильности)

In [ ]:
print("📊 Тестирование множественных запросов...\n")

symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN']
results = {}

for i, symbol in enumerate(symbols, 1):
    print(f"📈 {i}/{len(symbols)} Запрос данных для {symbol}...")
    
    try:
        data = tv.get_hist(symbol, 'NASDAQ', interval=Interval.in_daily, n_bars=5)
        
        if data is not None:
            results[symbol] = {
                'success': True,
                'records': len(data),
                'last_price': data['close'].iloc[-1]
            }
            print(f"   ✅ Успешно: {len(data)} записей, цена: ${data['close'].iloc[-1]:.2f}")
        else:
            results[symbol] = {'success': False, 'error': 'No data returned'}
            print(f"   ❌ Данные не получены")
            
    except Exception as e:
        results[symbol] = {'success': False, 'error': str(e)}
        print(f"   ❌ Ошибка: {e}")
    
    # Небольшая пауза между запросами
    time.sleep(1)

# Сводка результатов
print("\n📋 Сводка результатов:")
successful = sum(1 for r in results.values() if r['success'])
print(f"✅ Успешных запросов: {successful}/{len(symbols)}")
print(f"❌ Неудачных запросов: {len(symbols) - successful}/{len(symbols)}")

if successful > 0:
    print("\n💰 Последние цены:")
    for symbol, result in results.items():
        if result['success']:
            print(f"   {symbol}: ${result['last_price']:.2f}")

print("\n" + "="*50)

## 10. Очистка и заключение

In [ ]:
print("🧹 Очистка тестовых файлов...\n")

# Показываем содержимое файла токена
if os.path.exists(token_file):
    print(f"📄 Содержимое файла {token_file}:")
    try:
        with open(token_file, 'r', encoding='utf-8') as f:
            import json
            token_data = json.load(f)
            for key, value in token_data.items():
                if key == 'token':
                    print(f"   {key}: {value[:20]}...")
                else:
                    print(f"   {key}: {value}")
    except Exception as e:
        print(f"   ❌ Ошибка чтения файла: {e}")
    
    print(f"💾 Файл {token_file} сохранен для дальнейшего использования")

print("\n🎉 Демонстрация завершена!")
print("\n📋 Результаты тестирования:")
print("✅ Система управления токенами работает корректно")
print("✅ Автоматическое сохранение и загрузка токенов функционирует")
print("✅ Обработка истекших токенов реализована")
print("✅ Принудительное обновление токенов работает")
print("✅ Множественные запросы обрабатываются стабильно")
print("\n🚀 Система готова к использованию в продакшене!")